In [1]:
# %matplotlib inline
import sys
import os
sys.path.append(os.path.abspath('..')) # add parent folder to path
import backtrader as bt
import pandas as pd
import matplotlib
# matplotlib.use('QT5Agg')

In [2]:
dataname='../data/Temp.csv'

cerebro = bt.Cerebro()
df = pd.read_csv(dataname, index_col=0)
df.index = pd.to_datetime(df.index)
# df

In [3]:
from data_feeds.CoT_Dissag import DataFeedForAI
from strategies.AIStrategy import AIStrategy

data = DataFeedForAI(dataname=df)

cerebro.addstrategy(AIStrategy)

cerebro.adddata(data)

# 初始资金 100,000,000
cerebro.broker.setcash(2e3)
# 佣金，双边各 0.0003
cerebro.broker.setcommission(commission=0.0003)
# 滑点：双边各 0.0001
cerebro.broker.set_slippage_perc(perc=0.0001)

cerebro.addobserver(bt.observers.BuySell) # 买卖交易点
cerebro.addobserver(bt.observers.Value) # 价值


cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='pnl') # 返回收益率时序数据
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn') # 年化收益率
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio') # 夏普比率
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown') # 回撤

In [4]:
# 启动回测
result = cerebro.run()
cerebro.plot(iplot= False)
# 从返回的 result 中提取回测结果
strat = result[0]
# 返回日度收益率序列
daily_return = pd.Series(strat.analyzers.pnl.get_analysis())
# 打印评价指标
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())

init
self.data0:
('close', 'low', 'high', 'open', 'volume', 'openinterest', 'datetime', 'Col_1', 'Col_2', 'Col_3', 'Col_4', 'Col_5', 'Col_6', 'Col_7')
Bt starting...
2023-01-24, SELL EXECUTED, Price: 1931.80, Cost: -1545.44, Comm 0.46
2023-01-30, BUY EXECUTED, Price: 1928.49, Cost: -1545.44, Comm 0.46
2023-01-30, SELL EXECUTED, Price: 1928.11, Cost: -1542.49, Comm 0.46
2023-01-30, OPERATION PROFIT, GROSS 2.65, NET 1.72
2023-01-31, BUY EXECUTED, Price: 1923.19, Cost: -1542.49, Comm 0.46
2023-01-31, BUY EXECUTED, Price: 1923.19, Cost: 1538.55, Comm 0.46
2023-01-31, OPERATION PROFIT, GROSS 3.93, NET 3.01
2023-02-01, SELL EXECUTED, Price: 1927.01, Cost: 1538.55, Comm 0.46
2023-02-01, SELL EXECUTED, Price: 1927.01, Cost: -1541.61, Comm 0.46
2023-02-01, OPERATION PROFIT, GROSS 3.05, NET 2.13
2023-03-07, BUY EXECUTED, Price: 1840.90, Cost: -1541.61, Comm 0.44
2023-03-07, BUY EXECUTED, Price: 1840.90, Cost: 1472.72, Comm 0.44
2023-03-07, OPERATION PROFIT, GROSS 68.89, NET 67.98
2023-03-10, SEL

--------------- AnnualReturn -----------------
OrderedDict([(2023, 0.08786732719884593)])
--------------- SharpeRatio -----------------
OrderedDict([('sharperatio', None)])
--------------- DrawDown -----------------
AutoOrderedDict([('len', 2), ('drawdown', 0.4028317876434178), ('moneydown', 8.800000000000182), ('max', AutoOrderedDict([('len', 47), ('drawdown', 3.652384682949612), ('moneydown', 79.28300356179398)]))])
